
# Assignment 4 for Course 1MS041
Make sure you pass the `# ... Test` cells and
 submit your solution notebook in the corresponding assignment on the course website. You can submit multiple times before the deadline and your highest score will be used.

---
## Assignment 4, PROBLEM 1
Maximum Points = 24


    This time the assignment only consists of one problem, but we will do a more comprehensive analysis instead.

Consider the dataset `Corona_NLP_train.csv` that you can get from the course website [git](https://github.com/datascience-intro/1MS041-2024/blob/main/notebooks/data/Corona_NLP_train.csv). The data is "Coronavirus tweets NLP - Text Classification" that can be found on [kaggle](https://www.kaggle.com/datasets/datatattle/covid-19-nlp-text-classification). The data has several columns, but we will only be working with `OriginalTweet`and `Sentiment`.

1. [3p] Load the data and filter out those tweets that have `Sentiment`=`Neutral`. Let $X$ represent the `OriginalTweet` and let 
    $$
        Y = 
        \begin{cases}
        1 & \text{if sentiment is towards positive}
        \\
        0 & \text{if sentiment is towards negative}.
        \end{cases}
    $$
    Put the resulting arrays into the variables $X$ and $Y$. Split the data into three parts, train/test/validation where train is 60% of the data, test is 15% and validation is 25% of the data. Do not do this randomly, this is to make sure that we all did the same splits (we are in this case assuming the data is IID as presented in the dataset). That is [train,test,validation] is the splitting layout.

2. [4p] There are many ways to solve this classification problem. The first main issue to resolve is to convert the $X$ variable to something that you can feed into a machine learning model. For instance, you can first use [`CountVectorizer`](https://scikit-learn.org/1.5/modules/generated/sklearn.feature_extraction.text.CountVectorizer.html) as the first step. The step that comes after should be a `LogisticRegression` model, but for this to work you need to put together the `CountVectorizer` and the `LogisticRegression` model into a [`Pipeline`](https://scikit-learn.org/1.5/modules/generated/sklearn.pipeline.Pipeline.html#sklearn.pipeline.Pipeline). Fill in the variable `model` such that it accepts the raw text as input and outputs a number $0$ or $1$, make sure that `model.predict_proba` works for this. **Hint: You might need to play with the parameters of LogisticRegression to get convergence, make sure that it doesn't take too long or the autograder might kill your code**
3. [3p] Use your trained model and calculate the precision and recall on both classes. Fill in the corresponding variables with the answer.
4. [3p] Let us now define a cost function
    * A positive tweet that is classified as negative will have a cost of 1
    * A negative tweet that is classified as positive will have a cost of 5
    * Correct classifications cost 0
    
    complete filling the function `cost` to compute the cost of a prediction model under a certain prediction threshold (recall our precision recall lecture and the `predict_proba` function from trained models). 

5. [4p] Now, we wish to select the threshold of our classifier that minimizes the cost, fill in the selected threshold value in value `optimal_threshold`.
6. [4p] With your newly computed threshold value, compute the cost of putting this model in production by computing the cost using the validation data. Also provide a confidence interval of the cost using Hoeffdings inequality with a 99% confidence.
7. [3p] Let $t$ be the threshold you found and $f$ the model you fitted (one of the outputs of `predict_proba`), if we define the random variable
    $$
        C = (1-1_{f(X)\geq t})Y+5(1-Y)1_{f(X) \geq t}
    $$
    then $C$ denotes the cost of a randomly chosen tweet. In the previous step we estimated $\mathbb{E}[C]$ using the empirical mean. However, since the threshold is chosen to minimize cost it is likely that $C=0$ or $C=1$ than $C=5$ as such it will have a low variance. Compute the empirical variance of $C$ on the validation set. What would be the confidence interval if we used Bennett's inequality instead of Hoeffding in point 6 but with the computed empirical variance as our guess for the variance?

In [1]:

import pandas as pd
import numpy as np
# Part 1

# Load the data from the file specified in the problem definition and make sure that it is loaded using
# the search path `data/Corona_NLP_train.csv`. This is to make sure the autograder and your computer have the same
# file path and can load the data correctly.

# Contrary to how many other problems are structured, this problem actually requires you to
# have X on the shape (n_samples, ) that is a 1-dimensional array. Otherwise it will cause a bunch
# of errors in the autograder or also in for instance CountVectorizer.

# Make sure that all your data is numpy arrays and not pandas dataframes or series.
df = pd.read_csv("./data/Corona_NLP_train.csv", encoding_errors='replace')
df = df[df.Sentiment != 'Neutral']
X = df.OriginalTweet.values
Y = (df.Sentiment == 'Positive').astype(int).values

nrows = X.shape[0]
train_rows = int(nrows * 0.6)
test_rows = int(nrows * 0.15)


X_train = X[:train_rows]
Y_train = Y[:train_rows]
X_test = X[train_rows: train_rows + test_rows]
Y_test = Y[train_rows: train_rows + test_rows]
X_valid = X[train_rows + test_rows: ]
Y_valid = Y[train_rows + test_rows: ]

In [2]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression

vectorizer = CountVectorizer()
pipeline = Pipeline([('text_vectorizer', vectorizer), ('logistic_regression',LogisticRegression(penalty='l1', solver='liblinear'))])
pipeline.fit(X_train, Y_train)

# Part 2

# Train a machine learning model or pipeline that can take the raw strings from X and predict Y=0,1 depending on the
# sentiment of the tweet. Store the trained model in the variable `model`.

model = pipeline


In [3]:

# Part 3

# Evaluate the model on the test set and calculate precision, and recall on both classes. Store the results in the
# variables `precision_0`, `precision_1`, `recall_0`, `recall_1`.
from sklearn.metrics import precision_score, recall_score

precision_0 = precision_score(Y_test, model.predict(X_test), pos_label=0)
precision_1 = precision_score(Y_test, model.predict(X_test), pos_label=1)
recall_0 = recall_score(Y_test, model.predict(X_test), pos_label=0)
recall_1 = recall_score(Y_test, model.predict(X_test), pos_label=1)

In [4]:

# Part 4


def cost(model,threshold,X,Y):
    # Hint, make sure that the model has a predict_proba method
    # think about how the decision is made based on the probabilities
    # and how the threshold can be used to make the decision.
    # For reference take a look at the lecture notes "Bayes classifier"
    # which contains how the decision is made based on the probabilities when the threshold is 0.5.
    
    # Fill in what is missing to compute the cost and return it
    # Note that we are interested in average cost
    preds = model.predict_proba(X)[:,1] > threshold

    cost_fn = ((preds != Y) & (Y == 1)).sum()
    cost_fp = ((preds != Y) & (Y == 0)).sum()
    
    
    return (cost_fn + 5*cost_fp)/len(preds)

In [5]:

# Part 5

# Find the optimal threshold for the model on the test set. Store the threshold in the variable `optimal_threshold`
# and the cost at the optimal threshold in the variable `cost_at_optimal_threshold` evaluated on the test set.

thresholds = np.arange(0,1,.01)
costs = [cost(model, t, X_test, Y_test) for t in thresholds]


optimal_threshold = thresholds[np.argmin(costs)]
cost_at_optimal_threshold = np.min(costs)

In [6]:

# Part 6

cost_at_optimal_threshold_valid = cost(model, optimal_threshold, X_valid, Y_valid)
epsilon = np.sqrt(-25*np.log(0.005)/(2*len(Y_valid)))
cost_interval_valid = (cost_at_optimal_threshold_valid - epsilon, cost_at_optimal_threshold_valid + epsilon)

assert(type(cost_interval_valid) == tuple)
assert(len(cost_interval_valid) == 2)

In [12]:

# Part 7
import scipy.optimize as so

def c(model, x, y, threshold):
    
    preds = model.predict_proba(X)[:,1] > threshold
    cost_fn = 1 * ((preds != Y) & (Y == 1))
    cost_fp = 5 * ((preds != Y) & (Y == 0))
    return cost_fn + cost_fp

variance_of_C = np.var(c(model, X_valid, Y_valid, optimal_threshold))

b = 5
n = len(X_valid)
h = lambda u: (1+u)*np.log(1+u)-u
alpha = 0.01


f = lambda epsilon: np.exp(-n*variance_of_C/b**2*h(b*epsilon/variance_of_C))-alpha/2
ans = so.fsolve(f,x0=0.001)
epsilon_solution = np.abs(ans[0])
 

interval_of_C = (cost_at_optimal_threshold_valid - (epsilon_solution), cost_at_optimal_threshold_valid + (epsilon_solution))

assert(type(interval_of_C) == tuple)
assert(len(interval_of_C) == 2)